Potrebno je u anaconda prompt-u instalirati xgboost s:<br>
<code>conda install -c anaconda py-xgboost</code>

### Priprema

In [1]:
# potrebni paketi
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# učitavanje svih značajki iz training skupa
features = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/merged_all_features.csv")

# učitavanje svih značajki iz testing skupa
features_test = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/merged_test_features.csv")

In [2]:
one_gram_features = features.iloc[:,2:258]
metadata_bytes_features = features.iloc[:,258:260]
entropy_features = features.iloc[:,260:462]
image_features = features.iloc[:,462:514]
string_length_features = features.iloc[:,514:630]
metadata_asm_features = features.iloc[:,630:632]
symbols_features = features.iloc[:,632:639]
opcode_features = features.iloc[:,639:732]
reg_features = features.iloc[:,732:758]
section_features = features.iloc[:,758:782]
dd_features = features.iloc[:,782:806]
api_features = features.iloc[:,806:1600]
key_features = features.iloc[:,1600:1695]

### XGboost

In [3]:
#XGboost

X=features.iloc[:,2:1695]

X_train, X_test, y_train, y_test = train_test_split( X, features['Class'], test_size=0.3, random_state=47)

y_train = [int(x-1) for x in y_train]
y_test = [int(x-1) for x in y_test]

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
#print(X_train)
#print(y_test)
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 9}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

bst = xgb.train(param, dtrain, num_round)
bst.dump_model('dump.raw.txt')

preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])

score_prec = precision_score(y_test, best_preds, average='macro')
print('Precision_score : ')
print(score_prec)

#print(preds)
#print(best_preds)

score_log = log_loss(y_test,preds,eps=1e-15)
print('Logloss : ')
print(score_log)

score_acc = accuracy_score(y_test, best_preds)
print('Accuracy : ')
print(score_acc)

Precision_score : 
0.9938086555017404
Logloss : 
0.023918640948757782
Accuracy : 
0.9944802207911684


### Rezultat sa svim značajkama: <br>
Precision_score : <br>
0.9938086555017404<br>
Logloss : <br>
0.023918640948757782<br>
Accuracy : <br>
0.9944802207911684<br>